In [ ]:
import os
from dotenv import load_dotenv
from enum import Enum
from pprint import pprint

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

# 환경 변수 로드 및 API 키 확인
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print("API Key 시작 문자:", OPENAI_API_KEY[:2])

# 감정 Enum 정의
class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"  

# Enum 출력 파서 생성
enum_parser = EnumOutputParser(enum=Sentiment)
format_instructions = enum_parser.get_format_instructions()

print("감정 분류 출력 형식:")
print(format_instructions)

# 프롬프트 정의
template = """
당신은 영화 리뷰 감정 분석 전문가입니다.
다음 영화 리뷰의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

리뷰: "{text}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "보통" 중 하나의 단어만 출력하세요
2. 설명, 이모지, 특수문자 없이 한 단어만 출력하세요

답변:"""

# 프롬프트 구성
prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

# 모델 설정 (Groq 사용)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 안정성 보완용 파서
fixing_parser = OutputFixingParser.from_llm(parser=enum_parser, llm=model)

# 테스트용 리뷰 목록
test_reviews = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
]

# 안전 감정 분석 함수
def safe_sentiment_analysis(text, use_fixing_parser=True):
    try:
        chain = prompt | model | (fixing_parser if use_fixing_parser else enum_parser)
        result = chain.invoke({"text": text})
        return result.value, None
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)}"
    except Exception as e:
        return None, f"일반 오류: {str(e)}"

# 실행 함수
def run_sentiment_analysis():
    print("\n=== 영화 리뷰 감정 분석 결과 ===")
    success = 0

    for i, review in enumerate(test_reviews, 1):
        print(f"\n{i}. 리뷰: {review}")
        result, error = safe_sentiment_analysis(review)

        if result:
            print(f" 감정 분석 결과: {result}")
            success += 1
        else:
            print(f" 오류 발생: {error}")
            print(" 기본 파서로 재시도...")
            retry_result, retry_error = safe_sentiment_analysis(review, use_fixing_parser=False)

            if retry_result:
                print(f" 재시도 결과: {retry_result} (기본 파서)")
                success += 1
            else:
                print(f" 재시도 실패: {retry_error}")

    print(f"\n=== 요약 ===")
    print(f"총 {len(test_reviews)}개 중 {success}개 성공 ({success/len(test_reviews)*100:.1f}%)")

# 실행
if __name__ == "__main__":
    try:
        run_sentiment_analysis()
    except Exception as e:
        print("실행 실패. API 키 설정 또는 네트워크 오류일 수 있습니다.")
        print(f"오류: {e}")


API Key 시작 문자: gs
감정 분류 출력 형식:
Select one of the following options: 긍정, 부정, 보통

=== 영화 리뷰 감정 분석 결과 ===

1. 리뷰: 이 영화 정말 재미없어요. 시간 낭비였습니다.
   ➤ 감정 분석 결과: 부정

2. 리뷰: 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
   ➤ 감정 분석 결과: 긍정

3. 리뷰: 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
   ➤ 감정 분석 결과: 보통

=== 요약 ===
총 3개 중 3개 성공 (100.0%)
